In [2]:
import numpy as np
from PIL import Image

In [42]:
P = 16 # Patch size, is it constant for all steps of the pyramid?
factor, N = 2, 2 # Resizing factor, steps in pyramid
alpha = 5 * 0.0001 # Value is not clear
T = 5 # Value is not clear

In [16]:
from numpy.lib.stride_tricks import as_strided

def get_image_patches(image, patch_size):
    assert len(image.shape) == 3, "Image should have dimensions (width, height, channel)."
    assert image.shape[2] == 3, "Image should have 3 channels."

    stride_w = image.strides[0]  # stride of width
    stride_h = image.strides[1]  # stride of height
    stride_c = image.strides[2]  # stride of channel

    width, height, _ = image.shape
    patch_width, patch_height = patch_size, patch_size

    assert width >= patch_width, "Patch size should be less than or equal to image width."
    assert height >= patch_height, "Patch size should be less than or equal to image height."

    num_patches_w = width - patch_width + 1
    num_patches_h = height - patch_height + 1

    patches = as_strided(image, shape=(num_patches_w, num_patches_h, patch_width, patch_height, 3),
                         strides=(stride_w, stride_h, stride_w, stride_h, stride_c))

    return patches

(104, 187, 64, 64, 3)


In [59]:
def compute_distance_between_images(list1, list2):
    assert len(list1.shape) == 4, "Images should have dimensions (num_images, height, width, channels)."
    assert len(list2.shape) == 4, "Images should have dimensions (num_images, height, width, channels)."

    num_images1, height1, width1, channels1 = list1.shape
    num_images2, height2, width2, channels2 = list2.shape

    assert height1 == height2 and width1 == width2 and channels1 == channels2, \
        "Images in both lists should have the same dimensions."

    mse_values = np.zeros((num_images1, num_images2))

    for i in range(num_images1):
        for j in range(num_images2):
            mse = np.mean((list1[i] - list2[j]) ** 2)
            mse_values[i, j] = mse

    return mse_values

In [64]:
deneme = np.array([[1, 2, 3], [4, 5, 6]])
deneme = deneme / (np.min(deneme, axis=1))
deneme

ValueError: operands could not be broadcast together with shapes (2,3) (2,) 

In [65]:
deneme.shape

(2, 3)

In [51]:
image = Image.open('people_on_the_beach.jpg')
C = len(image.getbands())
image_arr = np.asarray(image)
image_arr.shape

(167, 250, 3)

In [ ]:
patches = get_image_patches(image_arr, P)  # Divide the image into (10, 10, 3) patches

In [40]:
# list of smaller images
smaller_image_arrs = [np.asarray(image.resize((int(image.size[0]/(i * factor)), int(image.size[1]/(i * factor))))) for i in range(1, N+1)]
smaller_image_arrs.insert(0, np.asarray(image))
[i.shape for i in smaller_image_arrs]

[(167, 250, 3), (83, 125, 3), (41, 62, 3)]

In [60]:
#Nth step (coarsest scale)
noise = np.random.normal(size=smaller_image_arrs[-1].shape)
Q = smaller_image_arrs[-1] + noise # y_n+1
Q = get_image_patches(Q, P)
Q = Q.reshape(-1, P, P, C)
V = smaller_image_arrs[-1] # x_n
V = get_image_patches(V, P)
V = V.reshape(-1, P, P, C)
K = smaller_image_arrs[-1] # x_n+1
K = get_image_patches(K, P)
K = K.reshape(-1, P, P, C)

Q.shape, K.shape, V.shape

((1222, 16, 16, 3), (1222, 16, 16, 3), (1222, 16, 16, 3))

In [56]:
Image.fromarray(K[0, 0])

In [57]:
K = K.reshape(-1, P, P, C)
Image.fromarray(K[0])

In [61]:
compute_distance_between_images(Q, K)

array([[1.05782278e+00, 7.95402946e+01, 1.66501152e+02, ...,
        2.59710987e+03, 2.60783927e+03, 2.71075081e+03],
       [7.98980830e+01, 1.04149947e+00, 7.46295593e+01, ...,
        2.55562175e+03, 2.50034610e+03, 2.58296025e+03],
       [1.65812723e+02, 7.44854965e+01, 1.02907723e+00, ...,
        2.58862949e+03, 2.48033423e+03, 2.50203487e+03],
       ...,
       [2.59043168e+03, 2.55245075e+03, 2.58607718e+03, ...,
        1.11038031e+00, 8.58322769e+02, 1.95144337e+03],
       [2.60942167e+03, 2.49912495e+03, 2.47748985e+03, ...,
        8.57340899e+02, 1.11322563e+00, 9.04269472e+02],
       [2.70887475e+03, 2.58603040e+03, 2.49914570e+03, ...,
        1.94993651e+03, 9.02890340e+02, 1.12787113e+00]])

In [22]:
def completeness_similarity(Q, K, a):
    # Q, K are lists of patches (#patches, p, p)
    # a is the normalizing factor
    D = (Q - K) ** 2 # patches axis uzerinden
    return D / (a + np.min(D, axis=1)) # axis=1 mi


def gpnn(x, y):
    
    # x is x_n
    # y is y_n+1
    return prediction